In [66]:
from admcycles import *

In [2]:
def pslambdaclass(d, g, n):
    """
    Inefficient method to calculate pullback of pslambda through T map.
    Based on [CGR+22]
    """
    L = lambdaclass(d, g, n)
    i = 1
    while i < d + 1:
        # Construct V: [g - i, 1, 1, ..., 1]
        V = [g - i] + [1] * i

        # Construct H:
        # [[1, 2, ..., n + i], [n + i + 1], [n + i + 2], ..., [n + i + i]]
        H = [list(range(1, n + i + 1))] + [[n + i + k] for k in range(1, i + 1)]

        # Construct E: [(1, i+1), (2, i+2), ..., (i, i+i)]
        E = [(n+k,n+i+k) for k in range(1, i + 1)]

        # Construct A: [lambdaclass(i, g - i, i), fundclass(1, 1), ..., fundclass(1, 1)]
        A = [lambdaclass(d-i, g - i, n + i)] + [fundclass(1, 1)] * i

        # Accumulate into L
        L += (1/factorial(i))*StableGraph(V, H, E).boundary_pushforward(A) 
        ## Curiously factorial can be removed and Matt's identity still holds

        i += 1

    return L


In [3]:
def pslambdaclass_iterative(d, g, n):
    """
    Iterative pslambdaclass that works for general n.
    Reuses lists between iterations but initializes them using n.
    """
    L = lambdaclass(d, g, n)

    # i = 1 initial setup (use general formula with n)
    i = 1
    V = [g - i] + [1] * i
    H = [list(range(1, n + i + 1))] + [[n + i + k] for k in range(1, i + 1)]
    E = [(n + k, n + i + k) for k in range(1, i + 1)]
    A = [lambdaclass(d - i, g - i, n + i)] + [fundclass(1, 1)] * i

    L += (1 / factorial(i)) * StableGraph(V, H, E).boundary_pushforward(A)

    # subsequent iterations: update incrementally but maintain correct labeling using n
    for i in range(2, d + 1):
        # Update V
        V[0] = g - i
        V.append(1)

        # Append the new label to the main block H[0]
        H[0].append(n + i)

        # Rebuild singleton leg blocks to reflect the new i:
        # H[1:] = [[n+i+1], [n+i+2], ..., [n+2*i]]
        H[1:] = [[n + i + k] for k in range(1, i + 1)]

        # Recompute E for current i (cheap)
        E = [(n + k, n + i + k) for k in range(1, i + 1)]

        # Update A (first entry changes; then append fundclasses)
        A[0] = lambdaclass(d - i, g - i, n + i)
        A = [A[0]] + [fundclass(1, 1)] * i

        L += (1 / factorial(i)) * StableGraph(V, H, E).boundary_pushforward(A)

    return L


In [53]:
def pslambdaclass_term(d, g, n, i):
    """
    Return only the i-th term in the pslambdaclass_iterative expansion.
    i ranges from 1 to d.
    """
    if i < 0 or i > d:
        raise ValueError("i must satisfy 1 ≤ i ≤ d")
    
    if i == 0:
        return lambdaclass(d,g,n)
    
    else:
        # Construct V
        V = [g - i] + [1] * i

        # Construct H
        H = [list(range(1, n + i + 1))] + [[n + i + k] for k in range(1, i + 1)]

        # Construct E
        E = [(n + k, n + i + k) for k in range(1, i + 1)]

        # Construct A
        A = [lambdaclass(d - i, g - i, n + i)] + [fundclass(1, 1)] * i

        # Normalization factor
        norm = 1 / factorial(i)

        # Return only this term
        return norm * StableGraph(V, H, E).boundary_pushforward(A)

## WORKSPACE ##

g=3
d=g
n=1
i=0

V = [g - i] + [1] * i
H = [list(range(1, n + i + 1))] + [[n + i + k] for k in range(1, i + 1)]
E = [(n + k, n + i + k) for k in range(1, i + 1)]
A = [lambdaclass(d - i, g - i, n + i)] + [fundclass(1, 1)] * i
norm = 1 / factorial(i)

G1=norm * StableGraph(V, H, E).boundary_pushforward(A)

#G2=(1/6)*StableGraph([1,1,1,1],[[1,2,3,4],[5],[6],[7]],[(2,5),(3,6),(4,7)]).boundary_pushforward([lambdaclass(1,1,4),fundclass(1,1),fundclass(1,1),fundclass(1,1)])

#G1-G2

(G1-pslambdaclass_term(3,3,1,0)).is_zero()


True

In [4]:
from itertools import product

def OtherSide(d,g,n):
    """
    Build the 'other side' sum described in Cavalieri and Williams (2024).
    This is the right hand side of the equation 
    pslambda_tot*pslambda^dual_tot=RHS
    """

    result = fundclass(g, n)

    for i in range(1, d + 1):
        # Construct V: [g - i, 1, 1, ..., 1]
        V = [g - i] + [1] * i

        # Construct H: [[1,2,..., n+i], [n+i+1], [n+i+2], ..., [n+i+i]]
        H = [list(range(1, n + i + 1))] + [[n + i + k] for k in range(1, i + 1)]

        # Construct E: [(n+1,n+i+1), (n+2,n+i+2), ..., (n+i, n+2i)]
        E = [(n + k, n + i + k) for k in range(1, i + 1)]

        # Build StableGraph once
        G = StableGraph(V, H, E)

        # Normalization factor (preserve previous behavior)
        norm = 1 / factorial(i)

        # Expand the product into 2^i monomials via bit-vectors
        terms = list(product((0, 1), repeat=i))  # each term is a tuple of 0/1 length i

        # For each bit-vector build the corresponding A and accumulate
        for idx, bits in enumerate(terms, start=1):
            # coefficient sign = (-1)^{# of ones}
            coef = (-1) ** (i+sum(bits))

            # Build first entry: product of psiclasses for every bit==1
            # If no bits==1, first_entry is the multiplicative identity '1' by default.
            first_entry = fundclass(g-i,n+i)
            any_psi_in_first = False
            for k, b in enumerate(bits, start=1):
                if b == 1:
                    any_psi_in_first = True
                    # multiply into first_entry
                    # assumes psiclass(...) returns an object supporting multiplication
                    first_entry = first_entry * psiclass(n + k, g - i, n + i)

            # If your ring does not accept integer 1 as multiplicative identity,
            # replace the '1' above with the appropriate identity object, e.g. unitclass()

            # Build tail entries: psiclass(1,1,1) when bits[k]==0, else fundclass(1,1)
            tail = []
            for k, b in enumerate(bits, start=1):
                if b == 0:
                    tail.append(psiclass(1, 1, 1))
                else:
                    tail.append(fundclass(1, 1))

            # Full A vector: first entry then the i leg-entries
            A = [first_entry] + tail

            # Accumulate into result
            result += coef * norm * G.boundary_pushforward(A)

    return result


In [6]:
def total_lambda(g, n):
    """Compute (1 + λ₁ + λ₂ + ... + λ_g) using pslambdaclass."""
    result = fundclass(g, n)  # this represents '1'
    for d in range(1, g + 1):
        result += pslambdaclass_iterative(d, g, n)
    return result


def total_dual_lambda(g, n):
    """Compute (1 - λ₁ + λ₂ - λ₃ + ... + (-1)^g λ_g) using pslambdaclass."""
    result = fundclass(g, n)  # start with '1'
    for d in range(1, g + 1):
        sign = (-1) ** d
        result += sign * pslambdaclass_iterative(d, g, n)
    return result

In [ ]:
def conjecture_sum(d, g, n):
    """
    Computes the full conjectural sum:
    For i = 1..d:
      V = [g-i] + [1]*i
      H = [ [1..n+i], [n+i+1], ..., [n+2i] ]
      E = [(n+1,n+i+1), ..., (n+i,n+2i)]
      A = [ lambdaclass(g-i,g-i,n+i) * product_{k=n+1..n+i} psiclass(k,g-i,n+i),
            lambdaclass(1,1,1), ..., lambdaclass(1,1,1) ]
    Also includes i=0 term: lambdaclass(g,g,n).
    """
    result = lambdaclass(g, g, n)  # i=0 term

    for i in range(1, d + 1):
        # V
        V = [g - i] + [1] * i

        # H
        H = [list(range(1, n + i + 1))] + [[n + i + k] for k in range(1, i + 1)]

        # E
        E = [(n + k, n + i + k) for k in range(1, i + 1)]

        # First A entry: lambdaclass(g-i,g-i,n+i) * psiclass(n+1..n+i)
        first_entry = lambdaclass(g - i, g - i, n + i)
        for k in range(n + 1, n + i + 1):
            first_entry = first_entry * psiclass(k, g - i, n + i)

        # Remaining A entries: lambdaclass(1,1,1)
        A = [first_entry] + [lambdaclass(1, 1, 1)] * i

        result += 1/factorial(i)*StableGraph(V, H, E).boundary_pushforward(A)

    return result

## WORKSPACE ##

# g=4
# d=g
# n=1

# (lambdaclass(d,g,n)*OtherSide(d,g,n)-conjecture_sum(d,g,n)).is_zero()

: 

In [52]:

def conjecture_term(d, g, n, i):
    """
    Returns only the i-th term of the conjectural sum.
    If i=0, return lambdaclass(g,g,n).
    """
    if i == 0:
        return lambdaclass(g, g, n)
    if i < 0 or i > d:
        raise ValueError("i must satisfy 0 ≤ i ≤ d")

    V = [g - i] + [1] * i
    H = [list(range(1, n + i + 1))] + [[n + i + k] for k in range(1, i + 1)]
    E = [(n + k, n + i + k) for k in range(1, i + 1)]

    first_entry = lambdaclass(g - i, g - i, n + i)
    for k in range(n + 1, n + i + 1):
        first_entry = first_entry * psiclass(k, g - i, n + i)

    A = [first_entry] + [lambdaclass(1, 1, 1)] * i

    return StableGraph(V, H, E).boundary_pushforward(A)

## WORKSPACE ##

g=3
d=g
n=1
i=1

V = [g - i] + [1] * i
H = [list(range(1, n + i + 1))] + [[n + i + k] for k in range(1, i + 1)]
E = [(n + k, n + i + k) for k in range(1, i + 1)]

#G1=StableGraph(V,H,E).boundary_pushforward([lambdaclass(2,2,2),psiclass(1,1,1)^2])-StableGraph(V,H,E).boundary_pushforward([lambdaclass(2,2,2)*psiclass(2,2,2)^2,fundclass(1,1)])
#G2=StableGraph([1,1,1],[[1,2,3],[4],[5]],[(2,4),(3,5)]).boundary_pushforward([lambdaclass(1,1,3)*psiclass(3,1,3),fundclass(1,1),lambdaclass(1,1,1)])-StableGraph([1,1,1],[[1,2,3],[4],[5]],[(2,4),(3,5)]).boundary_pushforward([lambdaclass(1,1,3),fundclass(1,1),lambdaclass(1,1,1)*psiclass(1,1,1)])
#((G1+2*G2)-(conjecture_term(d,g,n,i)*(OtherSide(1,g,n)-OtherSide(0,g,n)))).is_zero()
##FALSE, 
##but of course it's false, I'm using conjecture term, not pslambda_term




True

In [ ]:
## WORKSPACE1 ##
# G1 = StableGraph([2,1],[[1,2],[3]],[(2,3)]).to_tautological_class()
# G2 = StableGraph([1,1,1],[[1,2,3],[4],[5]],[(2,4),(3,5)]).to_tautological_class()
# G3 = StableGraph([0, 1, 1, 1],[[1, 7, 2, 3], [6], [4], [5]],[(2, 4), (3, 5), (6, 7)]).to_tautological_class()
# G4 = StableGraph([1, 1, 1],[[1, 2, 3], [4], [5]],[(2, 4), (3, 5)]).boundary_pushforward([-psiclass(2,1,3),fundclass(1,1),fundclass(1,1)])+StableGraph([1, 1, 1],[[1, 2, 3], [4], [5]],[(2, 4), (3, 5)]).boundary_pushforward([fundclass(1,3),-psiclass(1,1,1),fundclass(1,1)])
# (((G1*G2)-G3)-2*G4).is_zero()
## Above shows that psiclasses on both edges is the same as psiclasses twice on one edge.

G1 = StableGraph([2,1],[[1,2],[3]],[(2,3)]).to_tautological_class()
G2 = StableGraph([2,1],[[1,2],[3]],[(2,3)]).boundary_pushforward([lambdaclass(2,2,2),lambdaclass(1,1,1)])

((lambdaclass(3,3,1)*G1)-G2).is_zero()

## Check whether lambda class on glu1 bottom distributes as expected over new elliptic tails

# g=3
# d=g
# n=1
# i=1

# V = [g - i] + [1] * i
# H = [list(range(1, n + i + 1))] + [[n + i + k] for k in range(1, i + 1)]
# E = [(n + k, n + i + k) for k in range(1, i + 1)]

# #Glu1 with lambda2 on the bottom
# G1=StableGraph(V,H,E).boundary_pushforward([lambdaclass(2,2,2),fundclass(1,1)])

# GLU1=StableGraph(V,H,E).to_tautological_class()

# #Non-transverse part of intersection
# G2=StableGraph(V,H,E).boundary_pushforward([lambdaclass(2,2,2)*(-psiclass(2,2,2)),fundclass(1,1)])+StableGraph(V,H,E).boundary_pushforward([lambdaclass(2,2,2),-psiclass(1,1,1)])

# #Same non transverse part w/o lambdaclasses (just to check)
# GG2=StableGraph(V,H,E).boundary_pushforward([-psiclass(2,2,2),fundclass(1,1)])+StableGraph(V,H,E).boundary_pushforward([fundclass(2,2),-psiclass(1,1,1)])

# #Transverse intersection of G1 w Glu1
# G3=StableGraph([1, 1, 1],[[1, 2, 3], [4], [5]],[(2, 4), (3, 5)]).boundary_pushforward([lambdaclass(1,1,3),fundclass(1,1),lambdaclass(1,1,1)])

# #CHECK above
# #((G1*GLU1)-(G2+G3)).is_zero()
# #True

# ## Now checking if pslambda term works as intended

# #Non-transverse part of intersection glu1(p0(lambda2))*glu1(psiSt-psiDot)
# G1=StableGraph(V,H,E).boundary_pushforward([lambdaclass(2,2,2),psiclass(1,1,1)^2])-StableGraph(V,H,E).boundary_pushforward([lambdaclass(2,2,2)*psiclass(2,2,2)^2,fundclass(1,1)])

# #G1 after cancelling the psidot^2
# GG1=-StableGraph(V,H,E).boundary_pushforward([lambdaclass(2,2,2)*psiclass(2,2,2)^2,fundclass(1,1)])

# #Transverse part of intersection glu1(p0(lambda2))*glu1(psiSt-psiDot)
# G2=StableGraph([1,1,1],[[1,2,3],[4],[5]],[(2,4),(3,5)]).boundary_pushforward([lambdaclass(1,1,3)*psiclass(3,1,3),fundclass(1,1),lambdaclass(1,1,1)])-StableGraph([1,1,1],[[1,2,3],[4],[5]],[(2,4),(3,5)]).boundary_pushforward([lambdaclass(1,1,3),fundclass(1,1),lambdaclass(1,1,1)*psiclass(1,1,1)])

# #G2 after cancelling psidot with lambda1
# GG2=StableGraph([1,1,1],[[1,2,3],[4],[5]],[(2,4),(3,5)]).boundary_pushforward([lambdaclass(1,1,3)*psiclass(3,1,3),fundclass(1,1),lambdaclass(1,1,1)])

# #Check if indeed product gives what I say as G1+G2

# ((GG1+GG2)-(pslambdaclass_term(d,g,n,i)*(OtherSide(1,g,n)-OtherSide(0,g,n)))).is_zero()

True

In [ ]:
## WORKSPACE2 ##

# Checking coefficients for glu2^2

g=4
d=g
n=1
i=2

V = [g - i] + [1] * i
H = [list(range(1, n + i + 1))] + [[n + i + k] for k in range(1, i + 1)]
E = [(n + k, n + i + k) for k in range(1, i + 1)]

GLU2=StableGraph(V,H,E).to_tautological_class()

# First term is a copy of glu2 with both edges bicolored. There's 4 possible colorings, 2 automorphisms so coefficient is indeed 2.

# For the second term there's 24 possible colorings and 3! automorphisms.

H2=StableGraph([1, 1, 1, 1],[[1, 7, 2, 3], [6], [4], [5]],[(2, 4), (3, 5), (6, 7)]).boundary_pushforward([-psiclass(2,1,4),fundclass(1,1),fundclass(1,1),fundclass(1,1)])+StableGraph([1, 1, 1, 1],[[1, 7, 2, 3], [6], [4], [5]],[(2, 4), (3, 5), (6, 7)]).boundary_pushforward([fundclass(1,4),-psiclass(1,1,1),fundclass(1,1),fundclass(1,1)])

# H2 is the graph with one pair of psiclasses on one edge. Claim is Term2-4H2 is zero.

#Term 1's components are built from this four graphs.
T11=StableGraph(V,H,E).boundary_pushforward([fundclass(2,3),psiclass(1,1,1),psiclass(1,1,1)])
T12=StableGraph(V,H,E).boundary_pushforward([psiclass(3,2,3),psiclass(1,1,1),fundclass(1,1)])
T13=StableGraph(V,H,E).boundary_pushforward([psiclass(2,2,3),fundclass(1,1),psiclass(1,1,1)])
T14=StableGraph(V,H,E).boundary_pushforward([psiclass(2,2,3)*psiclass(3,2,3),fundclass(1,1),fundclass(1,1)])

#Then Term3 is

T3=StableGraph([0, 1, 1, 1, 1],[[1, 7, 2, 3, 8], [6], [9], [4], [5]],[(2, 4), (3, 5), (6, 7), (8, 9)]).to_tautological_class()

#Actual computation

T2=GLU2^2-2*(T11+T12+T13+T14)-T3
(T2-4*H2).is_zero()
#True so indeed, just having one bicolored and multiplying gives the correct coeff.


True